## Define Operator Algebra

In [1]:
using LinearAlgebra

struct Operator  # Linear Matrix Operators from Matrices to Matrices (and the operator adjoint)
    op
    adj
    sym
end

## Operators
ℒ(A::Matrix)  = Operator(X->A*X   , X->A'*X, "ℒ$(size(A))"  )   # left multiply by A (X → AX)
ℛ(A::Matrix)  = Operator(X->X*A   , X->X*A', "ℛ$(size(A))")     # right multiply by A (X → XA)
ℋ(A::Matrix)  = Operator(X->X.*A  , X->X.*A, "ℋ$(size(A))")    # Hadamard product (elementwise product)
ℐ()  =          Operator(X->X      ,    X->X,    "I")     # identity operator
𝒪()  =          Operator(X->zero(X) , X->zero(X),"𝒪")# zero operator

import Base:  zero, show, adjoint, *, \, ∘, +, -
show(io::IO, M::Operator) = print(io, M.sym)  # pretty printing
zero(::Any) = 𝒪() # Let's make any undefined zero the 𝒪 operator
adjoint(A::Operator) = Operator(A.adj, A.op,  "("*A.sym*")'")
adjoint(B::Bidiagonal) = Bidiagonal(adjoint.(B.dv),adjoint.(B.ev),(B.uplo == 'U') ? :L : :U) # lower to upper
-(A::Operator) = Operator(X->-A.op(X), X->-A.adj(X),"-"*A.sym)
-(::typeof(𝒪), X::Matrix) = -X # 𝒪 - X should be -X
*(A::Operator, X::Matrix) = A.op(X)
\(ℐ::typeof(ℐ()), A::Matrix) = A
∘(A::Operator, B::Operator) = Operator(A.op ∘ B.op, B.adj ∘ A.adj, A.sym*"∘"*B.sym)
# We need [A;B]*C to somehow magically be [AC;BC]
*(M::Adjoint{Operator, Matrix{Operator}},v::Array) = M .* [v]
+(A::Array,x::Number)=A.+x

+ (generic function with 209 methods)

## Example

In [2]:
# Basic Test
B = [ 1 2; 3 4]
M = [10 1;1 10]
C = [ 2 5;4 6]
ℒ(M)

ℒ(2, 2)

In [3]:
ℒ(M) * [ 1 0 ;0 1]

2×2 Matrix{Int64}:
 10   1
  1  10

In [4]:
ℒ(M) * B 

2×2 Matrix{Int64}:
 13  24
 31  42

In [5]:
ℛ(M) * B 

2×2 Matrix{Int64}:
 12  21
 34  43

In [6]:
ℋ(M) * B

2×2 Matrix{Int64}:
 10   2
  3  40

In [7]:
tr( B'*(ℒ(M)*C) ), tr( (ℒ(M)'*B) *C)    # <B,ℒC>=<ℒ'B,C>

(522, 529)

In [8]:
B = [ 1 2; 3 4]
M = Bidiagonal( [ℐ(),ℐ(),ℐ()] , [ℒ(B),ℒ(B)], :L)
display(Matrix(M))

3×3 Matrix{Operator}:
 I        𝒪        𝒪
 ℒ(2, 2)  I        𝒪
 𝒪        ℒ(2, 2)  I

In [9]:
display(Matrix(M'))

3×3 Matrix{Operator}:
 (I)'  (ℒ(2, 2))'  𝒪
 𝒪     (I)'        (ℒ(2, 2))'
 𝒪     𝒪           (I)'

In [10]:
b = [ rand(2,2) for i=1:3]
x = M'\b
display(M'*x .- b)

3-element Vector{Matrix{Float64}}:
 [5.551115123125783e-16 5.551115123125783e-16; 8.881784197001252e-16 1.5543122344752192e-15]
 [0.0 -1.1102230246251565e-16; 1.1102230246251565e-16 0.0]
 [0.0 0.0; 0.0 0.0]

In [11]:
M = Bidiagonal( [ℐ(),ℐ(),ℐ()] , [ℒ(B),ℒ(B)], :L)
display(Matrix(M))

b = [ rand(2,2) for i=1:3]
display(b)
x = M'\b
display(M'*x .- b)
display(Matrix(M'))

x = M\b
M*x .- b

3×3 Matrix{Operator}:
 I        𝒪        𝒪
 ℒ(2, 2)  I        𝒪
 𝒪        ℒ(2, 2)  I

3-element Vector{Matrix{Float64}}:
 [0.7989756643763034 0.6994720043869942; 0.5311685550526586 0.01255536589723849]
 [0.31403510755864295 0.4690199335775955; 0.25807620647389595 0.09489516270599407]
 [0.6404782426750693 0.09882041926930252; 0.32987698776175767 0.6856711146275344]

3-element Vector{Matrix{Float64}}:
 [3.3306690738754696e-16 -2.220446049250313e-16; 4.440892098500626e-16 1.1102230246251565e-16]
 [0.0 0.0; 2.220446049250313e-16 1.1102230246251565e-16]
 [0.0 0.0; 0.0 0.0]

3×3 Matrix{Operator}:
 (I)'  (ℒ(2, 2))'  𝒪
 𝒪     (I)'        (ℒ(2, 2))'
 𝒪     𝒪           (I)'

3-element Vector{Matrix{Float64}}:
 [0.0 0.0; 0.0 0.0]
 [0.0 0.0; -2.220446049250313e-16 1.1102230246251565e-16]
 [0.0 2.220446049250313e-16; -3.3306690738754696e-16 7.771561172376096e-16]

## Simple neural net

In [12]:

using OffsetArrays

h(x) =   exp(-x) # sample activation function
h′(x) = -exp(-x)

function neural_net(params,X₀;h=h,h′= h′)
    T = Matrix{Float64}
    N = length(params)
    X = OffsetArray(Vector{T}(undef,N+1),0:N)   
    Δ = Vector{T}(undef, N)
    X[0] = X₀
    W = first.(params)
    B = last.(params)
    
    for i=1:N         
          X[i] =  h.(W[i]*X[i-1] .+ B[i])
          Δ[i] =  h′.(W[i]*X[i-1] .+ B[i])        
    end 
    X,Δ
end

neural_net (generic function with 1 method)

## Initialization

In [13]:
n = [5,4,3,1]  ## this contains [n₀...n_N]
k = 10 # batchsize
N = length(n)-1 #should be positive
init(sizes...) = 0.01randn(sizes...)
Ws_and_bs =[ [init(n[i+1],n[i]) , init(n[i+1])]  for i=1:N] # The second part of the pair is a vector here
X₀ = init(n[1],k)
y  =  init(n[end],k); #  y is what we will compare X_N against
X,δ = neural_net(Ws_and_bs,X₀) # This has all the X's and δ's

𝓁(x,y) = sum(abs2,x-y)/2 #loss
𝓁′(x,y) = x.-y;

X,δ = neural_net(Ws_and_bs,X₀) # Run the neural net

([[0.01536138736216281 -0.002314957826792751 … 0.020699928039031375 -0.001560105073194762; 0.010476390104891467 -0.008457502061483999 … -0.013546234943512097 -0.007066433661190142; … ; -0.006540327026442212 -0.006982291795113741 … -0.0030033981904397595 -0.001268490286921531; -0.0021095578949297763 -0.017616786901637062 … 4.75613262723907e-5 0.005913323893731996], [1.0289516925427873 1.029373895976944 … 1.0294730999166006 1.0292794488301014; 0.9996859250591988 1.000223016684299 … 0.9999180189433828 0.999824347831108; 0.9915757042160019 0.9910621032639018 … 0.9912983673217222 0.9913323742763019; 0.9979042086139535 0.9981963246818899 … 0.9979728157458967 0.9980208819551164], [0.9677247503863952 0.9677100387282893 … 0.9677104992857102 0.9677164621391825; 0.9815164201430002 0.9815027310748933 … 0.9815084605595764 0.9815112758336031; 0.9372514452476292 0.9372418807660071 … 0.9372460457704974 0.9372482889386496], [1.0155963713371663 1.0155962390209345 … 1.0155962817270692 1.0155963208462546]

In [14]:
# params: `W_i` and `b_i`s: x_{i+1} <- Wi*x_i .+ b_i
Ws_and_bs =[ [init(n[i+1],n[i]) , init(n[i+1],k)]  for i=1:N] # The second part of the pair is a vector here
X₀ = init(n[1],k)
y  =  init(n[end],k); #  y is what we will compare X_N against
Ws_and_bs

3-element Vector{Vector{Matrix{Float64}}}:
 [[-0.0005507048434324012 -0.002954605361634197 … -0.0021617393912444303 0.024105599491719233; -0.007201549906450708 -0.005798743711922766 … -0.005987007331756691 -0.0014967275206959386; 0.004667319144206612 0.008033548688638339 … -0.00920714285576457 -0.008944785950421051; -0.01843784492012828 0.01033725775087447 … -0.003364824898193327 -0.013716445373529408], [0.006114646866216294 -0.022051379823206648 … 0.013577035427275112 0.0053404827445425376; -0.013970326915209042 -0.012531482202913958 … 0.007638271152535179 0.0056419388939410176; 0.005198598185988518 -0.017987485974699912 … -0.0032390950096365134 0.0020429268392451933; -0.020519093399548428 0.004838396429833115 … -0.013609733039085345 -0.018637985797441248]]
 [[0.006978847243198999 -0.014090763339538242 0.0055360825342700435 -0.015170015288024133; -0.016542690293549257 -0.009798857449634619 0.0028839905998063694 0.006083334597206355; -0.007145565033504457 -0.0018287662567956081 0.00288

## Backward diff a neural net with operators

In [15]:
X,δ = neural_net(Ws_and_bs,X₀) # This has all the X's and δ's

## The diagonal matrix
M = Diagonal([ [ℋ(δ[i]) ∘ ℛ(X[i-1])  ℋ(δ[i])] for i=1:N])

## The lower triangular matrix (I-L)
ImL = Bidiagonal([ℐ() for i in 1:N], -[ℋ(δ[i]) ∘ ℒ(Ws_and_bs[i][1]) for i=2:N] , :L)

## gradient of the loss function
g = [ fill(𝒪,N-1) ; [𝓁′(X[N],y)] ] 

## The gradient
∇J = M' * (ImL' \ g)

3-element Vector{Matrix{Matrix{Float64}}}:
 [[1.0286819188787345e-5 3.215460079552901e-6 … 3.7573252772455435e-6 -1.0968426082959141e-5; 2.8435813156522987e-6 8.143338304044974e-7 … 1.0444310268757361e-6 -3.0119523610083738e-6; -1.1319060059331868e-6 -3.207423186095412e-7 … -4.013625057925409e-7 1.2031717005068582e-6; -4.758831779038058e-6 -1.7364278479442322e-6 … -1.795112884836171e-6 5.190770250494352e-6]; [0.0002672661438335817 0.0002894618065930421 … 0.0002733363906652754 0.00027699570537750677; 7.224155869596787e-5 8.16373604984508e-5 … 7.591731344627279e-5 7.420628475019315e-5; -2.8273221676613997e-5 -3.2072248197002204e-5 … -3.0368534427706324e-5 -2.9680145344976655e-5; -0.00013075903822587598 -0.00013153043379239058 … -0.00013182725987008416 -0.00013451248342336104];;]
 [[-0.04113640592180197 -0.04123927200642847 -0.04133843091458516 -0.041554851115670796; 0.12483598457302274 0.12512539755506355 0.12544251018283753 0.12606864679045796; 0.057074680540890134 0.05721224311936387 0

In [16]:
#∇Jfd is the gradient calculated with finite differences method
∇Jfd = Ws_and_bs*0
ϵ = Ws_and_bs*0
𝜀 = .0001
for i=1:length(Ws_and_bs), wb=1:2
    for j=1:length(ϵ[i][wb])
        ϵ[i][wb][j] = 𝜀
        ∇Jfd[i][wb][j] = (𝓁(neural_net(Ws_and_bs+ϵ,X₀)[1][N],y).-𝓁(neural_net(Ws_and_bs-ϵ,X₀)[1][N],y))/2𝜀
        ϵ[i][wb][j] = .0
  end
 end

In [17]:
flatten(J) = vcat((x->x[:]).(vcat(J...))...)

flatten (generic function with 1 method)

In [18]:
norm(flatten(∇J)-flatten(∇Jfd))

1.1874460779648005e-7